In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import Diagonal

n_qubits = 3          
N = 2**n_qubits
L = 1.0              
D = 1.0              
t = 1.0               
x = 1.0               

phi = x * D * t * (2 * np.pi / L)**2

j = np.arange(N) #from 0 to N-1

k_squared = np.where(j < N/2, j**2, (j - N)**2)

unitary_diag = np.exp(-1j * phi * k_squared)

qc = QuantumCircuit(n_qubits)

evolution_gate = Diagonal(unitary_diag)

qc.append(evolution_gate, range(n_qubits))

print("Diagonal Operator Entries:")
print(np.round(unitary_diag, 3))

print("\nCircuit Diagram:")
print(qc)

Diagonal Operator Entries:
[ 1.   +0.j    -0.207-0.978j  0.672-0.741j -0.954+0.301j -0.981+0.193j
 -0.954+0.301j  0.672-0.741j -0.207-0.978j]

Circuit Diagram:
     ┌───────────┐
q_0: ┤0          ├
     │           │
q_1: ┤1 Diagonal ├
     │           │
q_2: ┤2          ├
     └───────────┘


/tmp/ipykernel_84006/2677257778.py:22: DeprecationWarning: The class ``qiskit.circuit.library.generalized_gates.diagonal.Diagonal`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use DiagonalGate instead.
  evolution_gate = Diagonal(unitary_diag)


In [11]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import Diagonal
from qiskit.quantum_info import Statevector

# --- 1. SETUP PARAMETERS ---
n_qubits = 3
N = 2**n_qubits
L = 1.0

# Define the Hamiltonian (k^2) diagonal
j_indices = np.arange(N)
k_squared = np.where(j_indices < N/2, j_indices**2, (j_indices - N)**2)
Hamiltonian_diag = (2 * np.pi / L)**2 * k_squared

# --- 2. LCHS PARAMETERS ---
# NOTE: 0.5 * (exp(ix) + exp(-ix)) = cos(x). 
# This specific setup creates a Cosine of the operator, not a pure exponential decay.
# To get pure decay, you would need Gaussian coefficients (Hubbard-Stratonovich).
coeffs = [0.5, 0.5]      
times  = [0.1, -0.1]     

# --- 3. BUILD THE CIRCUIT ---
reg_a = QuantumRegister(1, 'ancilla') 
reg_s = QuantumRegister(n_qubits, 'system')
qc = QuantumCircuit(reg_a, reg_s)

# A) PREPARE (Ancilla Superposition)
qc.h(reg_a[0]) 

# B) SELECT (Controlled-Hamiltonian Simulation)
# We use list(reg_s) to ensure we target the system qubits correctly

# -- Construct U0 (triggered on 0) --
U0_diag = np.exp(-1j * Hamiltonian_diag * times[0])
gate_U0 = Diagonal(U0_diag).control(1, ctrl_state=0) 
qc.append(gate_U0, [reg_a[0]] + list(reg_s))  # <--- FIXED HERE

# -- Construct U1 (triggered on 1) --
U1_diag = np.exp(-1j * Hamiltonian_diag * times[1])
gate_U1 = Diagonal(U1_diag).control(1, ctrl_state=1) 
qc.append(gate_U1, [reg_a[0]] + list(reg_s))  # <--- FIXED HERE

# C) UN-PREPARE
qc.h(reg_a[0])

print("LCU Circuit Constructed Successfully.")
qc.draw() # Uncomment to see the diagram

LCU Circuit Constructed Successfully.


/tmp/ipykernel_84006/3968421268.py:37: DeprecationWarning: The class ``qiskit.circuit.library.generalized_gates.diagonal.Diagonal`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use DiagonalGate instead.
  gate_U0 = Diagonal(U0_diag).control(1, ctrl_state=0)
/tmp/ipykernel_84006/3968421268.py:42: DeprecationWarning: The class ``qiskit.circuit.library.generalized_gates.diagonal.Diagonal`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use DiagonalGate instead.
  gate_U1 = Diagonal(U1_diag).control(1, ctrl_state=1)


┌───┐┌─────────────┐┌─────────────┐┌───┐
 ancilla: ┤ H ├┤0            ├┤0            ├┤ H ├
          └───┘│             ││             │└───┘
system_0: ─────┤1            ├┤1            ├─────
               │  c_Diagonal ││  c_Diagonal │     
system_1: ─────┤2            ├┤2            ├─────
               │             ││             │     
system_2: ─────┤3            ├┤3            ├─────
               └─────────────┘└─────────────┘

In [15]:
from qiskit import QuantumCircuit
import numpy as np
from qiskit.circuit.library import QFTGate, StatePreparation
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit
import numpy as np
import matplotlib.pyplot as plt

n_qubits=3


qcQFT = QFTGate(n_qubits)
qcIQFT = QFTGate(n_qubits).inverse()


x = np.linspace(0,2**n_qubits-1,2**n_qubits)
print(x)
state = np.zeros(2**n_qubits)
state[3] = 1
state = state/np.linalg.norm(state,2)

stateprep = StatePreparation(state)
circuit = QuantumCircuit(n_qubits+1)
circuit.compose(stateprep, qubits=range(1,n_qubits+1), inplace=True)
circuit.append(qcQFT, range(1,n_qubits+1))

circuit.compose(qc,inplace=True)
circuit.append(qcIQFT, range(1,n_qubits+1))
circuit.draw()

final_state = Statevector.from_instruction(circuit)


[0. 1. 2. 3. 4. 5. 6. 7.]
